In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import *
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

import mne

# NB: add more subjects in the data folder in neurolib package
from neurolib.utils.loadData import Dataset
from neurolib.models.aln import ALNModel

from neurolib.utils import functions as func

import brainplot as bp

from neurolib.utils import atlases
from nilearn import plotting  
atlas = atlases.AutomatedAnatomicalParcellation2()
# AAL2 atlas is taken from here: https://github.com/cwatson/brainGraph
aal2_atlas = pd.read_csv("aal2_coords.csv")
coords = aal2_atlas.loc[atlas.cortex, ["x.mni", "y.mni", "z.mni"]].to_numpy()


from eeg_simulations import simulate_raw_eeg, find_peaks

In [ ]:
# for n, i in enumerate(np.array(atlas.node_names)[atlas.cortex].tolist()):
#     print(f"{n}: '',  # '{i}'")

In [ ]:
# from mne.datasets import sample
# import os.path as op
# data_path = sample.data_path()
# subjects_dir = op.join(data_path, 'subjects')
# subject = 'sample'
# meg_path = op.join(data_path, 'MEG', subject)

In [ ]:
# labels = mne.read_labels_from_annot(subject, 'aparc.a2009s', subjects_dir=subjects_dir, verbose=False)
 #[l.name for l in labels]

In [ ]:
ds = Dataset("gw")
# ds.Cmat = ds.Cmats[3]
# ds.Dmat = ds.Dmats[3]
model = ALNModel(Cmat=ds.Cmat, Dmat=ds.Dmat)

# add custom parameter for downsampling results
# 10 ms sampling steps for saving data, should be multiple of dt
model.params['save_dt'] = 10.0
model.params["tauA"] = 600.0
model.params["sigma_ou"] = 0.0
model.params["b"] = 20.0

model.params["Ke_gl"] = 300.0
model.params["mue_ext_mean"] = 0.2
model.params["mui_ext_mean"] = 0.1

# Sleep model from newest evolution October 2020
model.params["mue_ext_mean"] = 3.3202829454334535
model.params["mui_ext_mean"] = 3.682451894176651
model.params["b"] =  3.2021806735984186
model.params["tauA"] = 4765.3385276559875
model.params["sigma_ou"] = 0.36802952978628106
model.params["Ke_gl"] = 265.48075753153


model.params['dt'] = 0.1
model.params['duration'] = 10 * 60 * 1000 #ms
model.params["signalV"] = 80.0



# Run ALN model and save results

In [ ]:
# model.state_vars
model.output_vars += ['seem', 'seev', 'mufe']
model.run(bold=True, append=True)
for key, val in model.outputs.items():
    print(f"{key} {val.shape}")
np.save("simultaion_dataset/aln_model.npy", model.mufe)
# data = np.load("simultaion_dataset/aln_model.npy")
local_peaks, global_peaks = find_peaks(model, model.output)
np.save("simultaion_dataset/local_peaks.npy", local_peaks)
np.save("simultaion_dataset/global_peaks.npy", global_peaks)

In [ ]:
plt.figure(figsize=(10, 4))
plt.imshow(model.outputs['seem'], aspect="auto", extent=[0, model.t[-1] / 1000, model.params.N, 0],
             cmap="plasma") # clim=(0, 20),
# cbar = plt.colorbar(extend='max', fraction=0.046, pad=0.04)
# cbar.set_label("Rate $r_{exc}$ [Hz]")
plt.colorbar()
plt.ylabel("Node")
plt.xlabel("Time [s]")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.imshow(model.outputs['rates_exc'], aspect="auto", extent=[0, model.t[-1] / 1000, model.params.N, 0],
             cmap="plasma") # clim=(0, 20),
# cbar = plt.colorbar(xxtend='max', fraction=0.046, pad=0.04)
# cbar.set_label("Rate $r_{exc}$ [Hz]")
plt.colorbar()
plt.ylabel("Node")
plt.xlabel("Time [s]")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.imshow(model.outputs['mufe'], aspect="auto", extent=[0, model.t[-1] / 1000, model.params.N, 0],
             cmap="plasma") # clim=(0, 20),
# cbar = plt.colorbar(xxtend='max', fraction=0.046, pad=0.04)
# cbar.set_label("Rate $r_{exc}$ [Hz]")
plt.colorbar()
plt.ylabel("Node")
plt.xlabel("Time [s]")
plt.tight_layout()
plt.show()

# Run raw EEG simulation

In [ ]:
simulate_raw_eeg("/SCZ_sleep/simultaion_dataset/eeg_simulation.fif", 10)


# Read results of the simulation

In [ ]:
raw = mne.io.read_raw_fif("simultaion_dataset/eeg_simulation.fif")

# Plot raw

In [ ]:
%matplotlib qt5

mne.viz.plot_raw(raw, block=True, n_channels=60, lowpass=15, highpass=0.1)

# Create epochs

In [ ]:
events = mne.events_from_annotations(raw)
epochs = mne.Epochs(
    raw, events[0], events[1], tmin=-2, tmax=2,
    baseline=None, preload=True, reject_by_annotation=False,
    proj=False, picks='eeg')

# epochs['GP'].plot_image()
# epochs['LP'].plot_image()

# Create evokes

In [ ]:
# evoked_GP = epochs['GP'][7].average() 

evoked_GP = epochs['GP'].average()
evoked_LP = epochs['LP'].average()

# times = np.arange(-2, 2, 0.1)
# fig, anim = evoked_GP.animate_topomap(
#     times=times, ch_type='eeg', frame_rate=4, time_unit='s', blit=False, show=False);
# anim.save('GP_evoked.gif', writer='imagemagick')
# evoked_GP.plot_topo();

# Plot evoked

In [ ]:
evoked_GP.plot_topomap([-1.5, -1, -0.5, 0, 0.5, 1, 1.5]);

In [ ]:
%matplotlib inline
evoked_GP.filter(0.1, 10)
fig, ax = plt.subplots(figsize=(15, 5))
evoked_GP.plot(axes=ax);

In [ ]:
%matplotlib inline
for ep in range(len(epochs['GP'])):
    evoked_GP_one = epochs['GP'][ep].average()
    evoked_GP_one.filter(0.1, 10)
    fig, ax = plt.subplots(figsize=(10, 5))
    evoked_GP_one.plot(axes=ax)
    plt.show()

In [ ]:
%matplotlib inline
for ep in range(len(epochs['LP'])):
    evoked_LP_one = epochs['LP'][ep].average()
    evoked_LP_one.filter(0.1, 10)
    fig, ax = plt.subplots(figsize=(10, 5))
    evoked_LP_one.plot(axes=ax)
    plt.show()